# Final Project of the NLP 2024 Course

Slides: https://docs.google.com/presentation/d/1NbH4E2HKVHQlaW_ivKCyjpWuEJFvmz3bSKsX8fs67tA/edit#slide=id.g2d17364e0e4_0_34


## Environment Setup

Get your own huggingface access token via
https://huggingface.co/settings/tokens

And set up HF_TOKEN as a secret of Colab

In [1]:
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

## Using the pre-trained model

In [2]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata

driveDir = '/content/drive/MyDrive/1122自然語言處理/自然語言處理期末報告/Model/fine-tuned-model'



tokenizer = AutoTokenizer.from_pretrained(driveDir, trust_remote_code=True, token=userdata.get('HF_TOKEN'))
model = AutoModelForCausalLM.from_pretrained(driveDir, trust_remote_code=True)

In [4]:
if isinstance(model, str):
  print(111)
else:
  print(222)

if isinstance(tokenizer, str):
  print(111)
else:
  print(222)

222
222


In [5]:
"""Module to generate OpenELM output given a model and an input prompt."""
import os
import logging
import time
import argparse
from typing import Optional, Union
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM

from google.colab import userdata


# The following function is revised from https://huggingface.co/apple/OpenELM/blob/main/generate_openelm.py
def generate(
    prompt: str,
    model: Union[str, AutoModelForCausalLM],
    hf_access_token: str = None,
    tokenizer: Union[str, AutoTokenizer] = 'meta-llama/Llama-2-7b-hf',
    device: Optional[str] = None,
    max_length: int = 1024,
    assistant_model: Optional[Union[str, AutoModelForCausalLM]] = None,
    generate_kwargs: Optional[dict] = None,
) -> str:
    """ Generates output given a prompt.
    Args:
        prompt: The string prompt.
        model: The LLM Model. If a string is passed, it should be the path to
            the hf converted checkpoint.
        hf_access_token: Hugging face access token.
        tokenizer: Tokenizer instance. If model is set as a string path,
            the tokenizer will be loaded from the checkpoint.
        device: String representation of device to run the model on. If None
            and cuda available it would be set to cuda:0 else cpu.
        max_length: Maximum length of tokens, input prompt + generated tokens.
        assistant_model: If set, this model will be used for
            speculative generation. If a string is passed, it should be the
            path to the hf converted checkpoint.
        generate_kwargs: Extra kwargs passed to the hf generate function.
    Returns:
        output_text: output generated as a string.
        generation_time: generation time in seconds.
    Raises:
        ValueError: If device is set to CUDA but no CUDA device is detected.
        ValueError: If tokenizer is not set.
        ValueError: If hf_access_token is not specified.
    """
    if not device:
        if torch.cuda.is_available() and torch.cuda.device_count():
            device = "cuda:0"
            logging.warning(
                'inference device is not set, using cuda:0, %s',
                torch.cuda.get_device_name(0)
            )
        else:
            device = 'cpu'
            logging.warning(
                (
                    'No CUDA device detected, using cpu, '
                    'expect slower speeds.'
                )
            )

    if 'cuda' in device and not torch.cuda.is_available():
        raise ValueError('CUDA device requested but no CUDA device detected.')

    if not tokenizer:
        raise ValueError('Tokenizer is not set in the generate function.')

    if not hf_access_token:
        raise ValueError((
            'Hugging face access token needs to be specified. '
            'Please refer to https://huggingface.co/docs/hub/security-tokens'
            ' to obtain one.'
            )
        )

    if isinstance(model, str):
        checkpoint_path = model
        model = AutoModelForCausalLM.from_pretrained(
            checkpoint_path,
            trust_remote_code=True
        )
    else:
        model = model

    model.to(device).eval()
    if isinstance(tokenizer, str):
        tokenizer = AutoTokenizer.from_pretrained(
            tokenizer,
            token=hf_access_token,
        )
    else:
        tokenizer = tokenizer

    # Speculative mode
    draft_model = None
    if assistant_model:
        draft_model = assistant_model
        if isinstance(assistant_model, str):
            draft_model = AutoModelForCausalLM.from_pretrained(
                assistant_model,
                trust_remote_code=True
            )
        draft_model.to(device).eval()

    # Prepare the prompt
    tokenized_prompt = tokenizer(prompt)
    tokenized_prompt = torch.tensor(
        tokenized_prompt['input_ids'],
        device=device
    )

    tokenized_prompt = tokenized_prompt.unsqueeze(0)


    # Generate
    stime = time.time()
    output_ids = model.generate(
        tokenized_prompt,
        max_length=max_length,
        pad_token_id=0,
        assistant_model=draft_model,
        **(generate_kwargs if generate_kwargs else {}),
    )
    generation_time = time.time() - stime

    output_text = tokenizer.decode(
        output_ids[0][tokenized_prompt.shape[1]:].tolist(),
        skip_special_tokens=True
    )

    return output_text, generation_time

## Implement your main function here
The input `abstract` is a `str` that forms an abstract of a research paper.
Your function will be invoked for returning the **sentence(s)** from the `abstract` that show the **research methodology**.

In [6]:
def extract_sentence(abstract: str) -> str:
    prompt = "From the following abstract, extract the sentences that shows the methods of the research. Only the sentences from the abstract, no other information.\n\n\n```%s```" % abstract
    output_text, genertaion_time = generate(
        prompt=prompt,
        # model="apple/OpenELM-1_1B-Instruct",
        hf_access_token=userdata.get('HF_TOKEN'),
        tokenizer=tokenizer,
        model=model,
    )
    return output_text

Your function is expected to be used as follows.

In [7]:
abstract = """The reliability of self-labeled data is an important issue when the data are regarded as ground-truth for training and testing learning-based models.
This paper addresses the issue of false-alarm hashtags in the self-labeled data for irony detection.
We analyze the ambiguity of hashtag usages and propose a novel neural network-based model, which incorporates linguistic information from different aspects, to disambiguate the usage of three hashtags that are widely used to collect the training data for irony detection.
Furthermore, we apply our model to prune the self-labeled training data.
Experimental results show that the irony detection model trained on the less but cleaner training instances outperforms the models trained on all data."""

predicted = extract_sentence(abstract)
print(predicted)




```The reliability of self-labeled data is an important issue when the data are regarded as ground-truth for training and testing learning-based models.
This paper addresses the issue of false-alarm hashtags in the self-labeled data for irony detection.
We analyze the ambiguity of hashtag usages and propose a novel neural network-based model, which incorporates linguistic information from different aspects, to disambiguate the usage of three hashtags that are widely used to collect the training data for irony detection.
Furthermore, we apply our model to prune the self-labeled training data.
Experimental results show that the irony detection model trained on the less but cleaner training instances outperforms the models trained on all data.```


```The reliability of self-labeled data is an important issue when the data are regarded as ground-truth for training and testing learning-based models.
This paper addresses the issue of false-alarm hashtags in the self-labeled data for iron

## Evaluation

We will evaluate your module with a close testset.
The sentence returned by your function will be compared with a golden reference.
The evaluation metric is `ROUGE-L`, which measures the overlap ratio between a predicted output and a reference. The details will be introduced in class.

In [8]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=bb3a41994a037f5cbefb78af158ac4ac29dc4a59698937395d6a12c68dee9eca
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [9]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'])

In [10]:
reference = """We analyze the ambiguity of hashtag usages and propose a novel neural network-based model, which incorporates linguistic information from different aspects, to disambiguate the usage of three hashtags that are widely used to collect the training data for irony detection. Furthermore, we apply our model to prune the self-labeled training data."""

print(scorer.score(reference, predicted)['rougeL'].fmeasure)

0.16987179487179488


In [11]:
predicted

'\n\n\n```The reliability of self-labeled data is an important issue when the data are regarded as ground-truth for training and testing learning-based models.\nThis paper addresses the issue of false-alarm hashtags in the self-labeled data for irony detection.\nWe analyze the ambiguity of hashtag usages and propose a novel neural network-based model, which incorporates linguistic information from different aspects, to disambiguate the usage of three hashtags that are widely used to collect the training data for irony detection.\nFurthermore, we apply our model to prune the self-labeled training data.\nExperimental results show that the irony detection model trained on the less but cleaner training instances outperforms the models trained on all data.```\n\n\n```The reliability of self-labeled data is an important issue when the data are regarded as ground-truth for training and testing learning-based models.\nThis paper addresses the issue of false-alarm hashtags in the self-labeled d

In [12]:
def evaluate(foo):
    import urllib.request
    test = "https://www.cs.nccu.edu.tw/~hhhuang/courses/nlp2024/test2024.in"
    gold = "https://www.cs.nccu.edu.tw/~hhhuang/courses/nlp2024/test2024.gold"

    from rouge_score import rouge_scorer
    scorer = rouge_scorer.RougeScorer(['rougeL'])

    total = 0
    cnt = 0
    with urllib.request.urlopen(test) as testin, \
         urllib.request.urlopen(gold) as gold:
        for input, ref in zip(testin, gold):
            input = input.decode("utf-8")
            ref = ref.decode("utf-8")
            output = foo(input)
            score = scorer.score(ref, output)['rougeL'].fmeasure
            cnt += 1
            total += score
            print("Test case %d: %g" % (cnt, score))
    print("Overall: %g" % (total / cnt))
    return total / cnt

# As your working function is `extract_sentence`, so do evaluation with the following statement
evaluate(extract_sentence)

Test case 1: 0.278164


Test case 2: 0.319574


Test case 3: 0.312012


Test case 4: 0.200318


Test case 5: 0.123779


...

Test case 97: 0.33271


Test case 98: 0.0645161


Test case 99: 0.0820189


Test case 100: 0.422665
Overall: 0.180079


0.1800794876756535